# Spatial Analysis and Comparisons of LA Metro
This notebook will compare <a href="https://transitrichhousing.org">Sasha Aicken's Transit Rich Housing</a> analysis with work done in-house.

In [ ]:
import fiona
import pandas as pd, numpy as np, json
import geopandas as gpd
from geopandas import GeoDataFrame
from shapely.geometry import Point
from shapely.geometry import Polygon
from matplotlib import pyplot as plt

## Terner/CCI Analysis

In [ ]:
stops = pd.read_csv("LA-Metro/Metro - Los Angeles.csv")
hqt_stops = pd.read_csv("LA-Metro/Metro - Los Angeles (HQT).csv")
rail_stops = pd.read_csv("LA-Metro/Metro - Los Angeles Rail.csv")

In [ ]:
plt.clf()

plt.figure(figsize=(20,20))

plt.plot(stops['stop_lon'], stops['stop_lat'], 'ok', markersize=1)
plt.plot(hqt_stops['stop_lon'], hqt_stops['stop_lat'], 'ob', markersize=1.5)
plt.plot(rail_stops['stop_lon'], rail_stops['stop_lat'], 'or', markersize=3.5)

plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.title("LA Metro - HQT Bus and Rail")

plt.legend(["All Bus","Qualifying Bus", "Qualifying Rail"])

plt.show()

### GeoPandas

In [ ]:
crs = {'init': 'epsg:4326'}

In [ ]:
# add geometry

stops_geo = gpd.GeoSeries([Point(xy) for xy in zip(stops['stop_lon'], stops['stop_lat'])])
hqt_stops_geo = gpd.GeoSeries([Point(xy) for xy in zip(hqt_stops['stop_lon'], hqt_stops['stop_lat'])])
rail_stops_geo = gpd.GeoSeries([Point(xy) for xy in zip(rail_stops['stop_lon'], rail_stops['stop_lat'])])

hqt_stops_gpd = gpd.GeoDataFrame(hqt_stops, crs=crs, geometry=hqt_stops_geo)
rail_stops_gpd = gpd.GeoDataFrame(rail_stops, crs=crs, geometry=rail_stops_geo)

hqt_stops_gpd.crs = crs
rail_stops_gpd.crs = crs


In [ ]:
hqt_stops_gpd = gpd.GeoDataFrame(hqt_stops, crs=crs, geometry=hqt_stops_geo)
hqt_stops_gpd.head()

### Station Area Buffers
This portion of the Jupyter notebook contains the code used to calculate the size of buffers in decimal degrees (per crs). This is a crucial step in our analytical process, as takes the point feature of a transit stop and translates into a polygon. The station area polygons will be used for all future analysis and to determine the spatial impacts of the legislation.

Per <a href="https://leginfo.legislature.ca.gov/faces/billTextClient.xhtml?bill_id=201720180SB827">SB 827</a> Chapter 4.35.65918.5(i):
    
    “Transit-rich housing project” means a residential development project the parcels of which are all within a:
       - one-half mile radius of a major transit stop* or 
       - a one-quarter mile radius of a stop on a high-quality transit bus corridor.
<p style="font-size:80%;">* <i>Major transit stop</i> means a site containing an existing rail transit station, or a ferry terminal served by either bus or rail transit service</p>



In [ ]:
test_lat = hqt_stops['stop_lat'][0]
#print(test_lat)

km_in_deg = (111.11 * np.cos(np.deg2rad(test_lat)))
km_in_deg

In [ ]:
(1/111.11 * np.cos(np.deg2rad(test_lat)))*(km2mi*bufferSize)

In [ ]:
km2mi*bufferSize/km_in_deg

In [ ]:
km2mi = 1.60934

# BUS
bufferSize = 0.25 # in miles
hqt_stops_gpd["buffer_dd"] = (km2mi*bufferSize)/(111.11 * np.cos(np.deg2rad(hqt_stops['stop_lat'])))

# RAIL
bufferSize = 0.5 # in miles
rail_stops_gpd['buffer_dd_hm'] = (km2mi*bufferSize)/((111.11 * np.cos(np.deg2rad(rail_stops_gpd['stop_lat']))))

bufferSize = 0.25 # in miles
# rail_stops_gpd['buffer_dd_qm'] = 1/((111.11 * np.cos(np.deg2rad(hqt_stops['stop_lat'])))/(km2mi*bufferSize))
rail_stops_gpd['buffer_dd_qm'] = (km2mi*bufferSize)/(111.11 * np.cos(np.deg2rad(hqt_stops['stop_lat']))) 
#rail_stops_gpd['buffer_dd_qm'] = (1/111.11 * np.cos(np.deg2rad(rail_stops_gpd['stop_lat'])))*(km2mi*bufferSize) 



In [ ]:
rail_stops_gpd.head()

In [ ]:
# create station-area buffers

# Bus
hqt_stops_buffers = gpd.GeoDataFrame(hqt_stops_gpd)
hqt_stops_buffers.crs = crs

hqt_stops_buffers['geometry'] = hqt_stops_buffers.apply(lambda x: x.geometry.buffer(x.buffer_dd), axis=1)
hqt_stops_buffers['dissolve_dummy'] = np.zeros(len(hqt_stops_buffers)) # value to use in dissolve


# Rail half-mile buffers
rail_stops_hm_buffers = gpd.GeoDataFrame(rail_stops_gpd)
rail_stops_hm_buffers.crs = crs

rail_stops_hm_buffers['geometry'] = rail_stops_hm_buffers.apply(lambda x: x.geometry.buffer(x.buffer_dd_hm), axis=1)
rail_stops_hm_buffers['dissolve_dummy'] = np.zeros(len(rail_stops_hm_buffers)) # value to use in dissolve

# Rail quarter-mile buffers
rail_stops_qm_buffers = gpd.GeoDataFrame(rail_stops_gpd)
rail_stops_qm_buffers.crs = crs

rail_stops_qm_buffers['geometry'] = rail_stops_qm_buffers.apply(lambda x: x.geometry.buffer(x.buffer_dd_qm), axis=1)
rail_stops_qm_buffers['dissolve_dummy'] = np.zeros(len(rail_stops_qm_buffers)) # value to use in dissolve

# dissolve
hqt_dissolve = hqt_stops_buffers[['dissolve_dummy', 'geometry']].dissolve(by="dissolve_dummy")
rail_hm_dissolve = rail_stops_hm_buffers[['dissolve_dummy', 'geometry']].dissolve(by="dissolve_dummy")
rail_qm_dissolve = rail_stops_qm_buffers[['dissolve_dummy', 'geometry']].dissolve(by="dissolve_dummy")

# Assign Coordinate Reference Systems to dissolved GPD

hqt_dissolve.crs = crs
rail_hm_dissolve.crs = crs
rail_qm_dissolve.crs = crs

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
#plt.xlim(-118.5,-118)
#plt.ylim(33.7,34.2)
ax = hqt_dissolve.plot(ax=ax, color='none', edgecolor='k', linewidth=0.5)
plt.plot(hqt_stops['stop_lon'], hqt_stops['stop_lat'], 'or')
# ax = rail_dissolve.plot(ax=ax, color='blue', edgecolor='k', alpha=0.3)
#our_layer2.plot(ax=ax, color='yellow', edgecolor='k', alpha=0.5)

plt.legend(["Bus"])
plt.title("LA Metro - Impacted")
plt.xlabel("Longitude")
plt.ylabel("Latitude")

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
#plt.xlim(-118.5,-118)
#plt.ylim(33.7,34.2)
ax = rail_hm_dissolve.plot(ax=ax, color='none', edgecolor='k', linewidth=0.5)
ax = rail_qm_dissolve.plot(ax=ax, color='none', edgecolor='r', linewidth=0.5)

#plt.plot(hqt_stops['stop_lon'], hqt_stops['stop_lat'], 'or')
# ax = rail_dissolve.plot(ax=ax, color='blue', edgecolor='k', alpha=0.3)
#our_layer2.plot(ax=ax, color='yellow', edgecolor='k', alpha=0.5)

plt.legend(["Bus"])
plt.title("LA Metro - Impacted")
plt.xlabel("Longitude")
plt.ylabel("Latitude")

plt.show()

## Transit Rich Housing
<i>This code was taken and adapted from Jared Nolan's https://github.com/simonhochberg/Upzoning/blob/master/Transit_Rich_Housing_comp.ipynb.</i>

In [ ]:
with open('transit-rich-housing/no_rise_shape.json') as json_data:
    no_rise_data = json.load(json_data)
with open('transit-rich-housing/low_rise_shape.json') as json_data:
    lo_rise_data = json.load(json_data)
with open('transit-rich-housing/high_rise_shape.json') as json_data:
    hi_rise_data = json.load(json_data)

In [ ]:
no_rise_trh_shapes = gpd.GeoDataFrame()

for j in list(range(1,len(no_rise_data))):
    lat = []
    lon = []
    for i in list(range(0,len(no_rise_data[j]))):
        lat.append(no_rise_data[j][i]['lat'])
        lon.append(no_rise_data[j][i]['lng'])

    poly1 = list(zip(lon, lat))

    shape = gpd.GeoSeries([Polygon(poly1)])
    shape = gpd.GeoDataFrame({'geometry': shape,"crs":crs})
    
    
    
    no_rise_trh_shapes = gpd.GeoDataFrame(pd.concat([no_rise_trh_shapes, shape], ignore_index=True))
    
lo_rise_trh_shapes = gpd.GeoDataFrame()

for j in list(range(1,len(lo_rise_data))):
    lat = []
    lon = []
    for i in list(range(0,len(lo_rise_data[j]))):
        lat.append(lo_rise_data[j][i]['lat'])
        lon.append(lo_rise_data[j][i]['lng'])

    poly1 = list(zip(lon, lat))

    shape = gpd.GeoSeries([Polygon(poly1)])
    shape = gpd.GeoDataFrame({'geometry': shape,"crs":crs})
    
    
    
    lo_rise_trh_shapes = gpd.GeoDataFrame(pd.concat([lo_rise_trh_shapes, shape], ignore_index=True))

hi_rise_trh_shapes = gpd.GeoDataFrame()

for j in list(range(1,len(hi_rise_data))):
    lat = []
    lon = []
    for i in list(range(0,len(hi_rise_data[j]))):
        lat.append(hi_rise_data[j][i]['lat'])
        lon.append(hi_rise_data[j][i]['lng'])

    poly1 = list(zip(lon, lat))

    shape = gpd.GeoSeries([Polygon(poly1)])
    shape = gpd.GeoDataFrame({'geometry': shape,"crs":crs})
    
    
    
    hi_rise_trh_shapes = gpd.GeoDataFrame(pd.concat([hi_rise_trh_shapes, shape], ignore_index=True))

In [ ]:
# Assign Coordinate Reference Systems

no_rise_trh_shapes.crs = crs
lo_rise_trh_shapes.crs = crs
hi_rise_trh_shapes.crs = crs

In [ ]:
fig, ax = plt.subplots(figsize=(20,20))
plt.xlim(-118.5,-118)
plt.ylim(33.7,34.2)
ax = no_rise_trh_shapes.plot(ax=ax, color='yellow', edgecolor='k', alpha=0.5)
ax = lo_rise_trh_shapes.plot(ax=ax, color='blue', edgecolor='k', alpha=0.5)
ax = hi_rise_trh_shapes.plot(ax=ax, color='green', edgecolor='k', alpha=0.5)
#our_layer2.plot(ax=ax, color='yellow', edgecolor='k', alpha=0.5)

plt.legend(["Bus"])
plt.title("Transit Rich Housing")
plt.xlabel("Longitude")
plt.ylabel("Latitude")

plt.show()

## Comparison

In [ ]:
fig, ax = plt.subplots(figsize=(20,20))
plt.xlim(-118.5,-118)
plt.ylim(33.7,34.2)
#ax = no_rise_trh_shapes.plot(ax=ax, color='yellow', edgecolor='k', alpha=0.5)
#ax = lo_rise_trh_shapes.plot(ax=ax, color='blue', edgecolor='k', alpha=0.5)
ax = hi_rise_trh_shapes.plot(ax=ax, color='blue', edgecolor='k', alpha=0.5)
# ax = lo_rise_trh_shapes.plot(ax=ax, color='blue', edgecolor='k', alpha=0.5)
ax = rail_qm_dissolve.plot(ax=ax, color='red', edgecolor='k', alpha=0.5)
# ax = rail_hm_dissolve.plot(ax=ax, color='yellow', edgecolor='k', alpha=0.5)

#plt.plot(rail_stops_gpd['stop_lon'], rail_stops_gpd['stop_lat'], 'ok')
#our_layer2.plot(ax=ax, color='yellow', edgecolor='k', alpha=0.5)

plt.legend(["TRH", "SSH"])
plt.title("Transit Rich Housing vs. In-House\n1/4 mile by rail")
plt.xlabel("Longitude")
plt.ylabel("Latitude")

plt.show()

<p>In the above graph, <span style="color:blue"><b>blue circles</b></span> represent the quarter-mile buffers around rail stations as defined by <i>Transit-Rich Housing</i>.</p><br>
<p>In the above graph, <span style="color:red"><b>red circles</b></span> represent the quarter-mile buffers around rail stations as defined by <i>our own analysis</i>.</p><br>

In [ ]:
fig, ax = plt.subplots(figsize=(20,20))
plt.xlim(-118.5,-118)
plt.ylim(33.7,34.2)
#ax = no_rise_trh_shapes.plot(ax=ax, color='yellow', edgecolor='k', alpha=0.5)
#ax = lo_rise_trh_shapes.plot(ax=ax, color='blue', edgecolor='k', alpha=0.5)
ax = lo_rise_trh_shapes.plot(ax=ax, color='blue', edgecolor='k', alpha=0.5)
# ax = lo_rise_trh_shapes.plot(ax=ax, color='blue', edgecolor='k', alpha=0.5)
ax = rail_hm_dissolve.plot(ax=ax, color='red', edgecolor='k', alpha=0.5)
# ax = rail_hm_dissolve.plot(ax=ax, color='yellow', edgecolor='k', alpha=0.5)

#plt.plot(rail_stops_gpd['stop_lon'], rail_stops_gpd['stop_lat'], 'ok')
#our_layer2.plot(ax=ax, color='yellow', edgecolor='k', alpha=0.5)

plt.legend(["TRH", "SSH"])
plt.title("Transit Rich Housing vs. In-House\n1/2 mile by rail")
plt.xlabel("Longitude")
plt.ylabel("Latitude")

plt.show()

<p>In the above graph, <span style="color:blue"><b>blue circles</b></span> represent the half-mile buffers around rail stations as defined by <i>Transit-Rich Housing</i>.</p><br>
<p>In the above graph, <span style="color:red"><b>red circles</b></span> represent the half-mile buffers around rail stations as defined by <i>our own analysis</i>.</p><br>

In [ ]:
fig, ax = plt.subplots(figsize=(20,20))
plt.xlim(-118.5,-118)
plt.ylim(33.7,34.2)
#ax = no_rise_trh_shapes.plot(ax=ax, color='yellow', edgecolor='k', alpha=0.5)
#ax = lo_rise_trh_shapes.plot(ax=ax, color='blue', edgecolor='k', alpha=0.5)
#ax = no_rise_trh_shapes.plot(ax=ax, color='blue', edgecolor='k', alpha=0.5)
# ax = lo_rise_trh_shapes.plot(ax=ax, color='blue', edgecolor='k', alpha=0.5)
ax = hqt_dissolve.plot(ax=ax, color='red', edgecolor='k', alpha=0.5)
# ax = rail_hm_dissolve.plot(ax=ax, color='yellow', edgecolor='k', alpha=0.5)

#plt.plot(hqt_stops['stop_lon'], hqt_stops['stop_lat'], 'ok')
#our_layer2.plot(ax=ax, color='yellow', edgecolor='k', alpha=0.5)

plt.legend(["TRH", "SSH"])
plt.title("Transit Rich Housing vs. In-House\n1/4 mile by bus")
plt.xlabel("Longitude")
plt.ylabel("Latitude")

plt.show()

### Point-in-polygon analysis
I am using point-in-polygon analysis to figure out which stops we're counting that Transit-Rich Housing is excluding and vice versa. This will hopefully explain the discrepancies in our approaches and help us better validate our data.

In doing so, I will execute the following steps:

<ol>
    <li><b>Dissolve a Transit-Rich Housing layer into a single multi-polygon.</b>
<br>
<ul>
    <li>Add a dummy variable for use in the dissolve process.</li>
    <li>Perform a dissolve and create a 1-row GeoDataFrame</li>
    </ul> 
    </li>
<br>
<li><b>Use the within() function to create a DataFrame of indicating whether a stop is contained within the TRH layer.</b>
<br>    
<ul>
    <li>Store the geometry of the dissolved layer in a new variable: <i>poly</i></li>
    <li>Perform the within() function and save results to a new dataframe: <i>within_results</i></li>
    </ul>
    </li>
<br>
    <li><b>Join results of point-in-polygon analysis to <i>hqt_stops</i> DataFrame for future evaluation</b>
<br>    
<ul>
    <li>Identify relevant columns to save.</li>
    <li>Join <i>within_results</i> to <i>hqt_stops</i>.</li>
    <li>Plot</li>
    </ul>
    </li>
</ol>

In [ ]:
# STEP 1: dissolve a Transit-Rich Housing layer into a single multi-polygon

# add a dummy variable
no_rise_trh_shapes['dissolve_dummy'] = np.zeros(len(no_rise_trh_shapes)) # value to use in dissolve

# dissolve
no_rise_dissolve = no_rise_trh_shapes[pd.isnull(no_rise_trh_shapes['geometry']) == False].dissolve(by="dissolve_dummy")

In [ ]:
# STEP 2: using within() for point-in-polygon

# store geometry of TRH layer in a new variable
poly = no_rise_dissolve['geometry'][0]
poly

In [ ]:
# use within() to evaluate whether or not a stop is located within the transit-rich housing layer

within_results = pd.DataFrame(hqt_stops_gpd[['stop_id','geometry']].within(poly), columns=["within"])
within_results.head()

In [ ]:
cols2keep = ['stop_id', 'stop_name', 'stop_lon',
       'stop_lat', 'am_pk_dir0', 'am_pk_dir1', 'pm_pk_dir0', 'pm_pk_dir1',
       'wkdy_dir0', 'wkdy_dir1', 'sat_dir0', 'sat_dir1', 'sun_dir0',
       'sun_dir1','within']

In [ ]:
hqt_stops_comp = hqt_stops.join(within_results)[cols2keep]

In [ ]:
plt.clf()

plt.figure(figsize=(15,15))

plt.plot(hqt_stops_comp[hqt_stops_comp['within'] == True]['stop_lon'], hqt_stops_comp[hqt_stops_comp['within'] == True]['stop_lat'], "ob", markersize=0.7)
plt.plot(hqt_stops_comp[hqt_stops_comp['within'] == False]['stop_lon'], hqt_stops_comp[hqt_stops_comp['within'] == False]['stop_lat'], "or", markersize=0.7)

plt.legend(["Included (per TRH)", "Excluded (per TRH)"])

plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.title("Qualifying LA Metro Stops:\nComparison between Transit-Rich Housing and Terner")


plt.show()

In [ ]:
list(hqt_stops_comp['within'].value_counts() / len(hqt_stops_comp))

In [ ]:
plt.clf()

plt.figure(figsize=(10,10))

plt.pie(list(hqt_stops_comp['within'].value_counts() / len(hqt_stops_comp)), labels=['Not in TRH (59%)', 'In TRH (41%)'], colors=['r', 'b'])


plt.legend(["Not in Transit-Rich Housing", "In Transit-Rich Housing"])
plt.title("Comparing Our Results with Transit-Rich Housing")

plt.show()

In [ ]:
# export hqt_stops_comp to look into the individual stops

hqt_stops_comp.to_csv("output/LA Metro h")